Building RAG System with LangChain and ChromaDB

In [445]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [446]:
# Import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

# Import vectorstore
from langchain.vectorstores import Chroma

1. Create Sample Data

In [447]:
# Define a list of documents, each as a separate string
docs = [
    '''
    Python Programming Language
    
    🐍 **Python** is a high-level, interpreted programming language known for its simplicity and readability.  
    💡 It supports multiple programming paradigms, including procedural, object-oriented, and functional styles.  
    🚀 Widely used in web development, data science, automation, AI, and more.  
    📚 Python has a vast ecosystem of libraries and frameworks like Django, NumPy, and TensorFlow.  
    🌍 Its clear syntax and strong community make it ideal for beginners and professionals alike.
    ''',

    '''
    Introduction to Machine Learning
    
    🤖 **Machine Learning (ML)** is a branch of artificial intelligence that enables systems to learn from data without being explicitly programmed.  
    📈 It uses algorithms to identify patterns, make predictions, or improve performance over time.  
    🧠 ML models are trained on datasets and adjust themselves based on feedback or new inputs.  
    🔍 Common types include supervised, unsupervised, and reinforcement learning.  
    💬 Applications range from speech recognition and spam filtering to fraud detection and self-driving cars.  
    🛠️ Tools like Python, TensorFlow, and Scikit-learn are popular in ML development.  
    🌍 ML is transforming industries by automating decisions and uncovering insights from big data.
    ''',

    '''
    Django for Backend Development
    
    🌐 **Django** is a high-level Python web framework that enables rapid development of secure and maintainable websites.  
    ⚙️ It follows the Model-View-Template (MVT) architectural pattern to separate data, logic, and presentation.  
    🚀 Django comes with built-in features like authentication, admin interface, and URL routing.  
    🛡️ It emphasizes security, helping developers avoid common threats like SQL injection and cross-site scripting.  
    📦 With its "batteries-included" philosophy, Django provides everything needed to build full-stack web apps.  
    🔍 It's widely used for content management systems, social networks, and scientific platforms.  
    🌍 Django powers popular sites like Instagram, Pinterest, and Mozilla.
    ''',

    '''
    Introduction to Deep Learning
    
    🧠 **Deep Learning** is a subset of machine learning that uses neural networks with many layers to model complex patterns in data.  
    🔗 These layers mimic the human brain’s structure, enabling systems to learn hierarchical representations.  
    📚 It excels at tasks like image recognition, natural language processing, and speech translation.  
    🧪 Deep learning models require large datasets and powerful computing resources to train effectively.  
    🕸️ Popular architectures include Convolutional Neural Networks (CNNs) and Recurrent Neural Networks (RNNs).  
    🛠️ Frameworks like TensorFlow and PyTorch are commonly used to build deep learning applications.  
    🚀 Deep learning drives innovations in autonomous vehicles, medical diagnostics, and generative AI.
    '''
]

# Verify how many documents are in the list
print(len(docs))  

4


In [448]:
# Save sample documents to files
import tempfile

files_dir = tempfile.mkdtemp()

for i, doc in enumerate(docs):
    with open(f'{files_dir}/doc_{i+1}.txt', 'w', encoding='utf-8') as f:
        f.write(doc)

print(f'Documents created in: {files_dir}')

Documents created in: C:\Users\USER\AppData\Local\Temp\tmphm8n99hy


In [449]:
# Save sample documents to files
import tempfile

files_dir = tempfile.mkdtemp()

for i, doc in enumerate(docs):
    with open(f'doc_{i+1}.txt', 'w', encoding='utf-8') as f:
        f.write(doc)

print(f'Documents created in: {files_dir}')

Documents created in: C:\Users\USER\AppData\Local\Temp\tmp959ppbhd


2. Load Document

In [450]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# Load documents from directory

loader = DirectoryLoader(
    'data',
    glob='*.txt',
    loader_cls=TextLoader,
    loader_kwargs={'encoding': 'utf-8'}
)

documents = loader.load()

print(f'Loaded {len(documents)} documents')
print(f'\nFirst document preview:')
print(documents[0].page_content)

Loaded 4 documents

First document preview:

    Python Programming Language

    🐍 **Python** is a high-level, interpreted programming language known for its simplicity and readability.  
    💡 It supports multiple programming paradigms, including procedural, object-oriented, and functional styles.  
    🚀 Widely used in web development, data science, automation, AI, and more.  
    📚 Python has a vast ecosystem of libraries and frameworks like Django, NumPy, and TensorFlow.  
    🌍 Its clear syntax and strong community make it ideal for beginners and professionals alike.
    


In [451]:
documents

[Document(metadata={'source': 'data\\doc_1.txt'}, page_content='\n    Python Programming Language\n\n    🐍 **Python** is a high-level, interpreted programming language known for its simplicity and readability.  \n    💡 It supports multiple programming paradigms, including procedural, object-oriented, and functional styles.  \n    🚀 Widely used in web development, data science, automation, AI, and more.  \n    📚 Python has a vast ecosystem of libraries and frameworks like Django, NumPy, and TensorFlow.  \n    🌍 Its clear syntax and strong community make it ideal for beginners and professionals alike.\n    '),
 Document(metadata={'source': 'data\\doc_2.txt'}, page_content='\n    Introduction to Machine Learning\n\n    🤖 **Machine Learning (ML)** is a branch of artificial intelligence that enables systems to learn from data without being explicitly programmed.  \n    📈 It uses algorithms to identify patterns, make predictions, or improve performance over time.  \n    🧠 ML models are train

3. Splitting Document

In [452]:
# Initialize text splitter
test_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,          # chunk size for balanced context
    chunk_overlap=50,        # overlap to maintain continuity between chunks
    length_function=len,     # function used to measure chunk length in characters
    separators=['\n\n', '\n', '. ', ' ', '']   # clean text‑splitting order
)

chunks = test_splitter.split_documents(documents)
chunks

[Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Python Programming Language'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='🐍 **Python** is a high-level, interpreted programming language known for its simplicity and readability.  \n    💡 It supports multiple programming paradigms, including procedural, object-oriented, and functional styles.  \n    🚀 Widely used in web development, data science, automation, AI, and more.  \n    📚 Python has a vast ecosystem of libraries and frameworks like Django, NumPy, and TensorFlow.'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='🌍 Its clear syntax and strong community make it ideal for beginners and professionals alike.'),
 Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning'),
 Document(metadata={'source': 'data\\doc_2.txt'}, page_content='🤖 **Machine Learning (ML)** is a branch of artificial intelligence that enables systems to learn from data witho

In [453]:
print('Chunks')
print(f'Content: {chunks[0].page_content}')
print(f'Metadata: {chunks[0].metadata}')

Chunks
Content: Python Programming Language
Metadata: {'source': 'data\\doc_1.txt'}


4. Embedding Model

In [454]:
# Load OPENAI_API_KEY
openai_key = os.getenv('OPENAI_API_KEY')

In [455]:
text = 'LangChain makes it easy to use LLMs.'
embeddings = OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000002688EA0BA00>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000002688EA0B790>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [456]:
vector = embeddings.embed_query(text)
vector

[0.006602696608752012,
 0.021901967003941536,
 -0.03336246311664581,
 -0.04902467131614685,
 0.0063758594915270805,
 0.009234003722667694,
 -0.02099462039768696,
 -0.004065613728016615,
 -0.005562737118452787,
 -0.016569558531045914,
 0.0004052527074236423,
 -0.0028127767145633698,
 -0.0032298073638230562,
 0.019947681576013565,
 0.010567106306552887,
 0.024149397388100624,
 0.016681231558322906,
 0.0037271035835146904,
 0.012074698694050312,
 0.0042296345345675945,
 -0.022683681920170784,
 -0.019794130697846413,
 -0.0010809646919369698,
 -0.0335858091711998,
 -0.0031757154501974583,
 0.011956045404076576,
 0.03626597300171852,
 -0.02025478333234787,
 -0.03741062805056572,
 0.0025074193254113197,
 0.013142576441168785,
 0.008256860077381134,
 -0.027862541377544403,
 -0.014336087740957737,
 -0.013603230006992817,
 -0.003988838288933039,
 0.021343600004911423,
 0.0011559954145923257,
 0.02042229473590851,
 0.0020275721326470375,
 0.02518237754702568,
 0.009876126423478127,
 0.00283895013

Initialize ChromaDB Vector Store and Stores the chunks in Vector Representation

In [457]:
# Create a Chromadb vector store
persist_directory='./chroma_db_1'

# Initialize Chromadb with OpenAI embeddings
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory=persist_directory,
    collection_name='rag_collection'
)

print(f'Vector store created with {vectorstore._collection.count()} vectors')
print(f'Persisted to: {persist_directory}')

Vector store created with 141 vectors
Persisted to: ./chroma_db_1


Test Similarity Search

In [458]:
query = 'What is Python Programming Language?'

similar_docs = vectorstore.similarity_search(query, k=3)
similar_docs

[Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Python Programming Language'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Python Programming Language'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Python Programming Language')]

In [459]:
query1 = 'What is Machine Learning?'
similar_docs_1 = vectorstore.similarity_search(query1, k=3)
similar_docs_1

[Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning'),
 Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning'),
 Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning')]

In [460]:
print(f'Query: {query1}')
print(f'\nTop {len(similar_docs_1)} similar chunks')

for i, doc in enumerate(similar_docs_1):
    print(f'\n----- Chunk {i+1} ----------')
    print(doc.page_content)
    print(f'Source: {doc.metadata.get('source', 'Unknown')}')

Query: What is Machine Learning?

Top 3 similar chunks

----- Chunk 1 ----------
Introduction to Machine Learning
Source: data\doc_2.txt

----- Chunk 2 ----------
Introduction to Machine Learning
Source: data\doc_2.txt

----- Chunk 3 ----------
Introduction to Machine Learning
Source: data\doc_2.txt


Similarity Score

In [461]:
results_scores = vectorstore.similarity_search_with_score(query1, k=3)
results_scores

[(Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning'),
  0.17662833631038666),
 (Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning'),
  0.17662833631038666),
 (Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning'),
  0.17662833631038666)]

Initialize LLM, RAG chain, Prompt Template, Query the RAG System

In [462]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo'
)

In [463]:
response = llm.invoke('What is LLM and RAG in AI perspective?')
response

AIMessage(content='In the context of artificial intelligence, LLM stands for Language Model (LM) and RAG stands for Retrieval-Augmented Generation. \n\nLanguage Models are AI models that are trained on vast amounts of text data to understand and generate human language. They are used in a variety of natural language processing tasks such as text generation, machine translation, and speech recognition.\n\nRAG, on the other hand, is a specific framework for language understanding and generation that combines the capabilities of language modeling with a retrieval mechanism. This allows the model to retrieve relevant information from a large knowledge base to generate more contextually relevant responses in tasks such as question answering and natural language generation.\n\nOverall, LLM and RAG are both important techniques in AI that help improve the understanding and generation of human language.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_token

In [464]:
from langchain.chat_models.base import init_chat_model

llm = init_chat_model('openai:gpt-3.5-turbo')
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002688D95A690>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002688D95A810>, root_client=<openai.OpenAI object at 0x000002688FB6C5F0>, root_async_client=<openai.AsyncOpenAI object at 0x000002688FB6D220>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [465]:
llm.invoke('What is LLM and RAG in AI perspective?')

AIMessage(content='In AI perspective, LLM stands for Language Model and RAG stands for Retrieval-Augmented Generation. \n\nLanguage Models (LLMs) are algorithms that have been trained on vast amounts of text data to predict the next word in a sequence of text. They are designed to understand and generate human language, and have been used in a wide range of natural language processing tasks such as machine translation, text generation, question answering, and more.\n\nRetrieval-Augmented Generation (RAG) is a technique that combines language models with retrievers, which are algorithms that are able to retrieve relevant information from a large set of documents. By combining these two approaches, RAG is able to generate more contextually relevant and accurate responses to queries or prompts. This approach has been particularly effective in question answering tasks, where the model first retrieves relevant information from a large database of documents and then generates a response base

In [466]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [467]:
# Convert vectorstore into retriever

retriever = vectorstore.as_retriever(
    search_kwargs = {'k': 3} # Retrieve top 3 relevant chunks
)
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002688EB28B90>, search_kwargs={'k': 3})

In [468]:
from langchain_core.prompts import ChatPromptTemplate

# Create prompt template
system_prompt = '''
    You are an AI assistant for question and answering tasks.
    Use the following retrieved context to answer the question.
    If you don't know the answer, say you don't know.
    Use 4 sentences maximum and keep the answer concise.
    
    Context: {context}
'''

prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('human', '{input}')
])

prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\n    You are an AI assistant for question and answering tasks.\n    Use the following retrieved context to answer the question.\n    If you don't know the answer, say you don't know.\n    Use 4 sentences maximum and keep the answer concise.\n\n    Context: {context}\n"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [469]:
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create a document chain
document_chain = create_stuff_documents_chain(
    llm,
    prompt
)


document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\n    You are an AI assistant for question and answering tasks.\n    Use the following retrieved context to answer the question.\n    If you don't know the answer, say you don't know.\n    Use 4 sentences maximum and keep the answer concise.\n\n    Context: {context}\n"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002688D95A690>, async_client=<openai

In [470]:
from langchain.chains import create_retrieval_chain

# Create RAG Chain
rag_chain = create_retrieval_chain(
    retriever,
    document_chain
)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002688EB28B90>, search_kwargs={'k': 3}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\n    You are an AI assistant for question and answering tasks.\n    Use the following retrieved context to answer the question.\n

In [471]:
response = rag_chain.invoke({
    'input': 'Wht is machine learning?'
})
response

{'input': 'Wht is machine learning?',
 'context': [Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning'),
  Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning'),
  Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning')],
 'answer': 'Machine learning is a branch of artificial intelligence that focuses on developing algorithms and statistical models to allow computers to improve their performance on a specific task without being explicitly programmed. It involves using data to train these models and make predictions or decisions without human intervention. Machine learning can be categorized into supervised learning, unsupervised learning, and reinforcement learning, depending on the type of data available for training. Overall, machine learning enables computers to learn and adapt from experience to perform tasks more accurately.'}

In [472]:
response1 = rag_chain.invoke({
    'input': 'What is javascript?'
})
response1

{'input': 'What is javascript?',
 'context': [Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Python Programming Language'),
  Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Python Programming Language'),
  Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Python Programming Language')],
 'answer': "I don't know."}

In [473]:
response2 = rag_chain.invoke({
    'input': 'What is javascript?'
})
response2

{'input': 'What is javascript?',
 'context': [Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Python Programming Language'),
  Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Python Programming Language'),
  Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Python Programming Language')],
 'answer': "I don't know, as the context provided is about the Python Programming Language."}

In [474]:
response3 = rag_chain.invoke({
    'input': 'What is ReactJs?'
})
response3

{'input': 'What is ReactJs?',
 'context': [Document(metadata={'source': 'data\\doc_1.txt'}, page_content='🌍 Its clear syntax and strong community make it ideal for beginners and professionals alike.'),
  Document(metadata={'source': 'data\\doc_1.txt'}, page_content='🌍 Its clear syntax and strong community make it ideal for beginners and professionals alike.'),
  Document(metadata={'source': 'data\\doc_1.txt'}, page_content='🌍 Its clear syntax and strong community make it ideal for beginners and professionals alike.')],
 'answer': 'ReactJs is a JavaScript library for building user interfaces. Its clear syntax and strong community make it ideal for beginners and professionals alike. ReactJs allows developers to create interactive user interfaces efficiently. It is widely used in web development for its component-based structure and virtual DOM.'}

In [475]:
response4 = rag_chain.invoke({
    'input': 'Who is the present Nigeria president?'
})
response4

{'input': 'Who is the present Nigeria president?',
 'context': [Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning'),
  Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning'),
  Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning')],
 'answer': "I'm sorry, I do not have that information."}

In [476]:
#Function for querying modern RAG System

def query_rag_system(question):
    
    print(f'Question: {question}')
    print('*' * 80)
    
    result = rag_chain.invoke({
        'input': question
    })
    
    print(f'Answer: {result['answer']}')
    print('\nRetrieved Contact:')
    
    for i, doc in enumerate(result['context']):
        print(f'\n... Source {i+1}....')
        print(doc.page_content)
        
    return result

# Test queries
questions = [
    'What do you understand by Python Programming?',
    'List main types of machine learning',
    'What is  deep learning?',
    'What is django?'
]

for question in questions:
    result = query_rag_system(question)
    print('\n' + '=' * 100 + '\n')

Question: What do you understand by Python Programming?
********************************************************************************
Answer: Python is a high-level programming language known for its readability and simplicity which allows developers to write clear and logical code. It supports multiple programming paradigms like procedural, object-oriented, and functional programming. Python's extensive standard library provides built-in modules for a wide range of tasks, making it versatile for various applications. Its dynamic typing and automatic memory management contribute to its ease of use for both beginners and experienced programmers.

Retrieved Contact:

... Source 1....
Python Programming Language

... Source 2....
Python Programming Language

... Source 3....
Python Programming Language


Question: List main types of machine learning
********************************************************************************
Answer: The main types of machine learning are supervised

Create RAG Chain Using LCEL (LangChain Expression LAnguage)

In [477]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [478]:
# create a custom prompt
custom_prompt = ChatPromptTemplate.from_template(
    '''
        You are an AI assistant for question and answering tasks.
        Use the following retrieved context to answer the question.
        If you don't know the answer, say you don't know.
        Use 4 sentences maximum and keep the answer concise.
        
        Context: {context}
        
        Question: {question}
        
        Answer:
    '''
)
custom_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\n        You are an AI assistant for question and answering tasks.\n        Use the following retrieved context to answer the question.\n        If you don't know the answer, say you don't know.\n        Use 4 sentences maximum and keep the answer concise.\n\n        Context: {context}\n\n        Question: {question}\n\n        Answer:\n    "), additional_kwargs={})])

In [479]:
# Format the output documents for the prompt

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [480]:
# Build RAG using LCEL
rag_chain_lcel = (
    {
        'context': retriever | format_docs, 
        'question': RunnablePassthrough()
    }
    | custom_prompt
    | llm
    | StrOutputParser()
)

rag_chain_lcel

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002688EB28B90>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\n        You are an AI assistant for question and answering tasks.\n        Use the following retrieved context to answer the question.\n        If you don't know the answer, say you don't know.\n        Use 4 sentences maximum and keep the answer concise.\n\n        Context: {context}\n\n        Question: {question}\n\n        Answer:\n    "), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002688D95A690>, async_

In [481]:
rag_chain_lcel.invoke('What is LLM in AI perspective?')

'LLM stands for Large Language Model in the AI perspective. It refers to models that are trained on vast amounts of text data to understand and generate human-like language. LLMs are used for a variety of natural language processing tasks such as text generation, translation, and summarization. They have been a significant advancement in the field of AI in recent years.'

In [482]:
rag_chain_lcel.invoke('What is machine learning?')

'Machine learning is a subset of artificial intelligence that focuses on developing algorithms and statistical models to enable computers to learn from and make decisions or predictions based on data. It involves training computers to recognize patterns and make decisions without being explicitly programmed to do so. Machine learning algorithms use data to improve their performance over time through experience. This technology is widely used in various applications such as recommendation systems, image recognition, and natural language processing.'

Adding New Documents To Existing Vector Store

In [483]:
new_document = """
Computer vision 

Computer vision is a field of artificial intelligence that
enables computers to derive meaningful information from digital
images, videos, and other visual inputs. Its primary goal is to
replicate the complexity of the human visual system, allowing
machines to "see" and understand their surroundings. Instead of 
perceiving an image as a whole picture, computers analyze visual
data as vast arrays of numerical pixel values representing color 
and brightness. By utilizing deep learning models, particularly 
Convolutional Neural Networks (CNNs), the system learns to 
identify patterns, edges, and textures within these numbers.
"""

new_document

'\nComputer vision \n\nComputer vision is a field of artificial intelligence that\nenables computers to derive meaningful information from digital\nimages, videos, and other visual inputs. Its primary goal is to\nreplicate the complexity of the human visual system, allowing\nmachines to "see" and understand their surroundings. Instead of \nperceiving an image as a whole picture, computers analyze visual\ndata as vast arrays of numerical pixel values representing color \nand brightness. By utilizing deep learning models, particularly \nConvolutional Neural Networks (CNNs), the system learns to \nidentify patterns, edges, and textures within these numbers.\n'

In [484]:
new_doc = Document(
    page_content=new_document,
    metadata={'source': 'manual_addition', 'topic': 'computer_vision'}
)

new_doc

Document(metadata={'source': 'manual_addition', 'topic': 'computer_vision'}, page_content='\nComputer vision \n\nComputer vision is a field of artificial intelligence that\nenables computers to derive meaningful information from digital\nimages, videos, and other visual inputs. Its primary goal is to\nreplicate the complexity of the human visual system, allowing\nmachines to "see" and understand their surroundings. Instead of \nperceiving an image as a whole picture, computers analyze visual\ndata as vast arrays of numerical pixel values representing color \nand brightness. By utilizing deep learning models, particularly \nConvolutional Neural Networks (CNNs), the system learns to \nidentify patterns, edges, and textures within these numbers.\n')

In [485]:
# Split the documents
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
new_chunks = text_splitter.split_documents([new_doc])

new_chunks

[Document(metadata={'source': 'manual_addition', 'topic': 'computer_vision'}, page_content='Computer vision \n\nComputer vision is a field of artificial intelligence that\nenables computers to derive meaningful information from digital\nimages, videos, and other visual inputs. Its primary goal is to\nreplicate the complexity of the human visual system, allowing\nmachines to "see" and understand their surroundings. Instead of \nperceiving an image as a whole picture, computers analyze visual\ndata as vast arrays of numerical pixel values representing color \nand brightness. By utilizing deep learning models, particularly \nConvolutional Neural Networks (CNNs), the system learns to \nidentify patterns, edges, and textures within these numbers.')]

In [486]:
# Add new document to vectorstore

vectorstore.add_documents(new_chunks)

['6ba953af-f469-407c-8e21-783638791e5f']

In [487]:
print(f'Added {len(new_chunks)} new chunks to the vector store')
print(f'Total vectors now: {vectorstore._collection.count()}')

Added 1 new chunks to the vector store
Total vectors now: 142


In [488]:
# Query with the updated vector

new_question = 'What is the main concept of computer vision?'

result = query_rag_system(new_question)
result

Question: What is the main concept of computer vision?
********************************************************************************
Answer: The main concept of computer vision is to enable computers to understand and derive meaningful information from digital images and videos. It aims to replicate the complexity of the human visual system, allowing machines to "see" and comprehend their surroundings. By analyzing visual data as numerical pixel values, computers can identify patterns, edges, and textures within images. Deep learning models, especially Convolutional Neural Networks (CNNs), are commonly used in computer vision to achieve this goal.

Retrieved Contact:

... Source 1....
Computer vision 

Computer vision is a field of artificial intelligence that
enables computers to derive meaningful information from digital
images, videos, and other visual inputs. Its primary goal is to
replicate the complexity of the human visual system, allowing
machines to "see" and understand the

{'input': 'What is the main concept of computer vision?',
 'context': [Document(metadata={'topic': 'computer_vision', 'source': 'manual_addition'}, page_content='Computer vision \n\nComputer vision is a field of artificial intelligence that\nenables computers to derive meaningful information from digital\nimages, videos, and other visual inputs. Its primary goal is to\nreplicate the complexity of the human visual system, allowing\nmachines to "see" and understand their surroundings. Instead of \nperceiving an image as a whole picture, computers analyze visual\ndata as vast arrays of numerical pixel values representing color \nand brightness. By utilizing deep learning models, particularly \nConvolutional Neural Networks (CNNs), the system learns to \nidentify patterns, edges, and textures within these numbers.'),
  Document(metadata={'topic': 'computer_vision', 'source': 'manual_addition'}, page_content='Computer vision \n\nComputer vision is a field of artificial intelligence that\nen

Advanced RAG Techniques: Conversational Memory

In [489]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [490]:
# Create a prompt with addition of chat history
contextualize_q_system_prompt = '''
    Given a chat history and the latest user question
    which might reference context in the chat history, formulate a standalone question
    which can be understood without the chat history. Do not answer the question,
    just reformulate it if number or otherwise return it as it.
'''

# Create chat prompt
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ('system', contextualize_q_system_prompt),
    MessagesPlaceholder('chat_history'),
    ('human', '{input}'),
])

In [491]:
# Create history aware retriever

history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    contextualize_q_prompt
)

history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002688EB28B90>, search_kwargs={'k': 3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(ta

In [492]:
# Create a new document chain with history

qa_system_prompt = '''
    You are an AI assistant for question and answering tasks.
    Use the following retrieved context to answer the question.
    If you don't know the answer, say you don't know.
    Use 4 sentences maximum and keep the answer concise.
    
    Context: {context}
'''

qa_prompt = ChatPromptTemplate.from_messages([
    ('system', qa_system_prompt),
    MessagesPlaceholder('chat_history'),
    ('human', '{input}'),
])

question_answer_chain = create_stuff_documents_chain(
    llm,
    qa_prompt
)

# Create concersational RAG chain
conversational_rag_chain = create_retrieval_chain(
    history_aware_retriever,
    question_answer_chain
)

print('Conversational RAG chain created successfully')

Conversational RAG chain created successfully


In [493]:
# 1st Question
chat_history = []
result_1 = conversational_rag_chain.invoke({
    'chat_history': chat_history,
    'input': 'What is machine learning?'
})

print(f'Q: What is machine learning?')
print(f'A: {result_1['answer']}')

Q: What is machine learning?
A: Machine learning is a branch of artificial intelligence that focuses on developing systems that can learn from data and improve over time without being explicitly programmed. It involves algorithms that enable computers to make decisions or predictions based on patterns in data. Machine learning is used in a wide range of applications, from recommendation systems to image recognition. Overall, it is about creating models that can learn from and make predictions or decisions based on data.


In [494]:
chat_history

[]

In [495]:
chat_history.extend([
    HumanMessage(content='What is machine learning?'),
    AIMessage(content=result_1['answer'])
])

In [496]:
chat_history

[HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Machine learning is a branch of artificial intelligence that focuses on developing systems that can learn from data and improve over time without being explicitly programmed. It involves algorithms that enable computers to make decisions or predictions based on patterns in data. Machine learning is used in a wide range of applications, from recommendation systems to image recognition. Overall, it is about creating models that can learn from and make predictions or decisions based on data.', additional_kwargs={}, response_metadata={})]

In [497]:
# Next question
result_2 = conversational_rag_chain.invoke({
    'chat_history': chat_history,
    'input': 'What are its main?'
})

result_2

{'chat_history': [HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Machine learning is a branch of artificial intelligence that focuses on developing systems that can learn from data and improve over time without being explicitly programmed. It involves algorithms that enable computers to make decisions or predictions based on patterns in data. Machine learning is used in a wide range of applications, from recommendation systems to image recognition. Overall, it is about creating models that can learn from and make predictions or decisions based on data.', additional_kwargs={}, response_metadata={})],
 'input': 'What are its main?',
 'context': [Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning'),
  Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction to Machine Learning'),
  Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Introduction t

In [498]:
result_2['answer']

"Machine learning's main goal is to develop models that can analyze data, learn from it, and make predictions or decisions. It involves training algorithms to recognize patterns in data so they can make accurate predictions on new, unseen data. Machine learning also focuses on optimizing models to improve their performance over time and adapting to new data. Overall, its main purpose is to enable computers to learn from data and make decisions or predictions without being explicitly programmed."

Build RAG Chain with LCEL Using GROQ

In [499]:
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")

# llm = init_chat_model(
#     model="gemma2-27b-it",
#     model_provider="groq",
#     api_key=groq_api_key
# )

llm = ChatGroq(
    model="llama3-8b-8192",
    api_key=os.getenv("GROQ_API_KEY")
)


llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002688FB6CD70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002688FB6E5D0>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [500]:
# Basic RAG Chain with LCEL

simple_prompt = ChatPromptTemplate.from_template(
    '''
    Answer the question base only on the following context:
    Context: {context}
    
    Question: {question}
    
    Answer:
    '''
)

In [501]:
# Create basic retriever

retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 3}
)
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002688EB28B90>, search_kwargs={'k': 3})

In [502]:
from typing import List

# Format documents for the prompt

def format_docs(docs: List[Document]) -> str:
    '''Format documents for insertion into prompt'''
    
    formatted = []
    
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f'Document {i+1} (source: {source}):\ndoc.page_content')
        
    return '\n\n'.join(formatted)

In [503]:
basic_rag_chain = (
    {
        'context': retriever | format_docs,
        'question': RunnablePassthrough()
    }
    | simple_prompt
    | llm
)
basic_rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002688EB28B90>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n    Answer the question base only on the following context:\n    Context: {context}\n\n    Question: {question}\n\n    Answer:\n    '), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002688FB6CD70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002688FB6E5D0>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [504]:
# Create Conversational Rag Chain

conversational_prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful AI assistant. Use the provided context to answer the questions.'),
    ('placeholder', '{chat_history}'),
    ('human', 'Context: {context}\n\nQuestion: {input}')
])

In [505]:
def create_conversational_rag():
    '''Create a conversational RAG chain with memory'''
    return (
        RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x['input']))
        )
        | conversational_prompt
        | llm
        | StrOutputParser()
    )
    
conversational_rag = create_conversational_rag()
conversational_rag

RunnableAssign(mapper={
  context: RunnableLambda(lambda x: format_docs(retriever.invoke(x['input'])))
})
| ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag=

In [506]:
# Tet function for different chain types

def test_rag_chain(question: str):
    
    '''Test all RAG chains'''
    print(f'Question: {question}')
    print('-' * 90)
    
    print('\n#1. Basic RAG Chain')
    answer = basic_rag_chain.invoke(question)
    print(f'Answer: {answer}')

In [507]:
test_rag_chain('Differentiate between machine learning and deep learning')

Question: Differentiate between machine learning and deep learning
------------------------------------------------------------------------------------------

#1. Basic RAG Chain


BadRequestError: Error code: 400 - {'error': {'message': 'The model `llama3-8b-8192` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}